# --- Day 4: Security Through Obscurity ---

Finally, you come across an information kiosk with a list of rooms. Of course, the list is encrypted and full of decoy data, but the instructions to decode the list are barely hidden nearby. Better remove the decoy data first.

Each room consists of an encrypted name (lowercase letters separated by dashes) followed by a dash, a sector ID, and a checksum in square brackets.

A room is real (not a decoy) if the checksum is the five most common letters in the encrypted name, in order, with ties broken by alphabetization. For example:

- `aaaaa-bbb-z-y-x-123[abxyz]` is a real room because the most common letters are a (5), b (3), and then a tie between x, y, and z, which are listed alphabetically.
- `a-b-c-d-e-f-g-h-987[abcde]` is a real room because although the letters are all tied (1 of each), the first five are listed alphabetically.
- `not-a-real-room-404[oarel]` is a real room.
- `totally-real-room-200[decoy]` is not.

Of the real rooms from the list above, the sum of their sector IDs is 1514.

**What is the sum of the sector IDs of the real rooms?**

In [173]:
# the puzzle input
with open('inputs/4.txt') as f:
    data = f.read().strip().split("\n")
data[:3]

['aczupnetwp-dnlgpyrpc-sfye-dstaatyr-561[patyc]',
 'jsehsyafy-vqw-ljsafafy-866[nymla]',
 'tyepcyletzylw-ncjzrpytn-prr-opawzjxpye-743[cnrdl]']

first up, lets seperate each line into its room name, sector_id and checksum.

In [92]:
import re
from collections import namedtuple, Counter

Room = namedtuple("Room", ["name", "id", "checksum"])

def parse_line(line):
    m = re.search("\d+", line) # find all the digits
    room_name = line[:m.span()[0]-1]
    sector_id = int(m[0])
    checksum = line[m.span()[1]+1:-1]
    return Room(room_name, sector_id, checksum)

parse_line(line)

Room(name='aczupnetwp-dnlgpyrpc-sfye-dstaatyr', id=561, checksum='patyc')

Now to parse all our rooms into a list:

In [93]:
rooms = [parse_line(room) for room in data]
rooms[:3]

[Room(name='aczupnetwp-dnlgpyrpc-sfye-dstaatyr', id=561, checksum='patyc'),
 Room(name='jsehsyafy-vqw-ljsafafy', id=866, checksum='nymla'),
 Room(name='tyepcyletzylw-ncjzrpytn-prr-opawzjxpye', id=743, checksum='cnrdl')]

So, first up I used `Counter(room_name).most_common(5)` but that failed as Counter breaks ties arbitarily. So I had to use most_common() and break ties first by number then alphabetically.

In [120]:
c = Counter(rooms[0].name.replace("-","")).most_common()
print(c[:10])
sorted(c, key=lambda x: (-x[1],x[0]), reverse=False)[:10]

[('p', 4), ('a', 3), ('t', 3), ('y', 3), ('c', 2), ('n', 2), ('e', 2), ('d', 2), ('r', 2), ('s', 2)]


[('p', 4),
 ('a', 3),
 ('t', 3),
 ('y', 3),
 ('c', 2),
 ('d', 2),
 ('e', 2),
 ('n', 2),
 ('r', 2),
 ('s', 2)]

Another thing I learned today was making the number to sort on negative, to sort it the way I wanted (since the numers should be sorted high to low, while alphabets are sorted from low to high).

In [137]:
def checksum(room):
    """takes in a room and returns the checksum"""
    checksum = []
    
    c = Counter(room.name.replace("-","")).most_common()
    # now to sort the key by descending count and ascending alphabet
    c.sort(key=lambda x: (-x[1],x[0]))
    
    for char, count in c[:5]:
        checksum.append(char)
        
    return "".join(checksum)

for room in rooms[:3]:
    print(room.checksum, checksum(room))

patyc patyc
nymla afsyj
cnrdl pyert


Now to sum up all the valid ids:

In [135]:
sum([room.id for room in rooms if checksum(room)==room.checksum])

278221

# --- Part Two ---

With all the decoy data out of the way, it's time to decrypt this list and get moving.

The room names are encrypted by a state-of-the-art shift cipher, which is nearly unbreakable without the right software. However, the information kiosk designers at Easter Bunny HQ were not expecting to deal with a master cryptographer like yourself.

To decrypt a room name, rotate each letter forward through the alphabet a number of times equal to the room's sector ID. A becomes B, B becomes C, Z becomes A, and so on. Dashes become spaces.

For example, the real name for `qzmt-zixmtkozy-ivhz-343` is very encrypted name.

**What is the sector ID of the room where North Pole objects are stored?**

First up, a function to rotate a letter:

In [149]:
import string

def rotate(l, num):
    letters = string.ascii_lowercase
    i = (letters.index(l) + num) % len(letters)
    return letters[i]

rotate("a", 27)

'b'

Now to decrypt. It could be all done in the one function, but this keeps it simple:

In [171]:
def decrypt(room):
    """returns the decrypted name of a room"""
    name = []
    
    for char in room.name:
        if char == "-":
            name.append(" ")
        else:
            name.append(rotate(char, room.id))
    
    return "".join(name)

decrypt(rooms[0])

'projectile scavenger hunt shipping'

I assume the room we are looking for will have northpole in it:

In [170]:
[(room, decrypt(room)) for room in rooms if "northpole" in decrypt(room)]

[(Room(name='ghkmaihex-hucxvm-lmhktzx', id=267, checksum='hmxka'),
  'northpole object storage')]

# Notes:

- there are lots of little tricks in these puzzles, like sorting the same list in two different orders
- namedtuples are great, much easier to read than a list
- is there a better way to build a string? Right now I append chars to a list then join them into a string.